In [ ]:
""" Summarization Training (Zero Shot Prompting) with Flan-T5-Base on Enriched MTS Dialogue Dataset
    https://huggingface.co/sarahahatee/t5-summarization-zero-shot-headers-and-better-prompt-base-enriched"""

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # only run if you gen an encoding error
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

NameError: name 'locale' is not defined

In [3]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install numpy
!pip install tensorflow
!pip install rouge_score
!pip install -U accelerate
!pip install transformers[torch]
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git
!pip install bleurt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8dac25fd801bdb2b6e21ed6655d16c22958150bf1f03f6408e8cb7c6699d6f43
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 574.6 kB/s eta 0:00:00
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-er642ndf
  Running command git clone --filter=blob:none --quiet https://github.com/google-r

In [4]:
import accelerate
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
)
import numpy as np
import torch._dynamo
import nltk
from huggingface_hub import notebook_login

notebook_login()


In [5]:
# Load training datasets
train_file_a = "/content/MTS-Dialog-TrainingSet-enriched-vectorized.csv"
# train_file_b = "data/MTS-Dialog-ValidationSet.csv"

# Load testing dataset
# test_file = "data/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv"
test_file = "/content/MTS-Dialog-TestSet-2-MEDIQA-Sum-2023.csv"

# Load training datasets from CSV files
train_dataset_a = load_dataset('csv', data_files=train_file_a)['train']
# train_dataset_b = load_dataset('csv', data_files=train_file_b)['train']

# Load testing dataset from CSV file
test_dataset = load_dataset('csv', data_files=test_file)['train']

# Combine the two training datasets
# combined_train_dataset = concatenate_datasets([train_dataset_a, train_dataset_b])
combined_train_dataset = train_dataset_a

# Create dialog_dataset dictionary
dialog_dataset = DatasetDict({
    "train": combined_train_dataset,
    "test": test_dataset
})
# loading flan-t5 instead of plain t5
checkpoint = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = checkpoint

# establish prefix
PREFIX_SUMMARIZE = "Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. This dialogue is classified as "

MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128


def preprocess_function(examples, include_umls_cui_mappings=False):
    """Preprocess inputs for summarization with optional inclusion of key UMLS CUI mappings."""

    if include_umls_cui_mappings and 'umls_cui_mappings' in examples:
        inputs = [f"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: {dialogue} [CUI Mappings: {cui_mappings}]\nSummary: "
                  for dialogue, cui_mappings in zip(examples["dialogue"], examples["umls_cui_mappings"])]
    else:
        inputs = [f"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: {dialogue}\nSummary: "
                  for dialogue in examples["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["section_text"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Example of processing with UMLS CUI mappings
tokenized_data_with_umls = dialog_dataset.map(
    lambda examples: preprocess_function(examples, include_umls_cui_mappings=True),
    batched=True
)

# Example output
tokenized_data_with_umls['train']
tokenizer.decode(tokenized_data_with_umls['train'][0]['input_ids'])

# get an example of preprocessed data.
# print(tokenized_data['train'][0])
# print(tokenizer.decode(tokenized_data['train'][0]['input_ids']))

# load_metric rouge is deprecated but evaluate.rouge requires different format of data
# rouge = evaluate.load("rouge")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: Doctor: What brings you back into the clinic today, miss? Patient: I came in for a refill of my blood pressure medicine. Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones. Have you noticed any changes or do you have any concerns regarding these issues? Patient: No. Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure? Patient: No. Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as? Patient: I am seventy six years old and identify as a white female. [CUI Mappings: [('patient', ['C0030705']), ('hypertension', ['C0020538', 'C0020538', 'C0020538']), ('history', ['C0262926']), ('hypertension', ['C0020538', 'C0020538', 'C0020538']), ('osteoarthritis', ['C

In [6]:
from bleurt import score as bleurt_score
from bert_score import score as bert_score
import evaluate


data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=checkpoint, label_pad_token_id=-100, pad_to_multiple_of=8
)

bleurt = evaluate.load('bleurt', config_name='BLEURT-20-D3')
# bleurt = load_metric("bleurt", module_type="metric", checkpoint="BLEURT-20")
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    """Prepare evaluation metrics - Rouge, BERTScore, BLEURT-20"""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge
    decoded_preds_rouge = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels_rouge = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    rouge_result = metric.compute(
        predictions=[decoded_preds_rouge],
        references=[decoded_labels_rouge],
        use_stemmer=True
    )
    rouge_result = {key: value.mid.fmeasure for key, value in rouge_result.items()}

    # BERTScore
    _, _, bert_scores = bert_score(decoded_preds, decoded_labels, lang="en", verbose=False)
    bert_f1 = bert_scores.mean().item()  # Take the mean BERTScore value

    # BLEURT-20
    scorer = bleurt_score.BleurtScorer(bleurt)
    bleurt_scores = scorer.score(references=decoded_labels, candidates=decoded_preds)
    mean_bleurt_score = np.mean(bleurt_scores)

    # Calculate mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    gen_len = np.mean(prediction_lens)


    return {
        'rouge': {k: round(v, 4) for k, v in rouge_result.items()},
        'bert_score': round(bert_f1, 4),
        'bleurt_20': round(mean_bleurt_score, 4),
        'gen_len': round(gen_len, 4)
    }


# load t5 model
model = checkpoint

# prepare trainer arguments. tweak if performance unsatisfactory.
BATCH_SIZE = 2
args = Seq2SeqTrainingArguments(
    "t5-summarization-zero-shot-headers-and-better-prompt-base-enriched",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    warmup_ratio=0.1,
    predict_with_generate=True,
    fp16=False,
    logging_steps=75,
    eval_steps=75,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data_with_umls["train"],
    eval_dataset=tokenized_data_with_umls["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


torch._dynamo.config.suppress_errors = True
# to make it continue if you don't have any GPU available

# don't know what it does but it's VITAL.
nltk.download("punkt")

trainer.train()

trainer.evaluate()

trainer.push_to_hub()


<ipython-input-6-b12cbf9098a2>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge,Bert Score,Bleurt 20,Gen Len
1,2.188000,2.100317,"{'rouge1': 0.4472, 'rouge2': 0.1969, 'rougeL': 0.1958, 'rougeLsum': 0.1958}",0.876600,-0.805000,14.265000
2,1.819700,1.966758,"{'rouge1': 0.4259, 'rouge2': 0.1977, 'rougeL': 0.2091, 'rougeLsum': 0.2091}",0.880300,-0.785400,13.395000
3,1.616000,1.927892,"{'rouge1': 0.4209, 'rouge2': 0.1984, 'rougeL': 0.2069, 'rougeLsum': 0.2069}",0.878800,-0.791500,13.385000
4,1.417400,1.960094,"{'rouge1': 0.4294, 'rouge2': 0.2009, 'rougeL': 0.2098, 'rougeLsum': 0.2098}",0.879600,-0.745300,13.745000
5,1.207300,1.969029,"{'rouge1': 0.3801, 'rouge2': 0.1813, 'rougeL': 0.2045, 'rougeLsum': 0.2045}",0.879300,-0.802400,12.630000
6,0.978000,2.102395,"{'rouge1': 0.4035, 'rouge2': 0.1887, 'rougeL': 0.2067, 'rougeLsum': 0.2067}",0.880200,-0.742700,13.080000
7,0.899400,2.129995,"{'rouge1': 0.4209, 'rouge2': 0.1962, 'rougeL': 0.2063, 'rougeLsum': 0.2063}",0.882100,-0.735100,13.315000
8,0.813300,2.218266,"{'rouge1': 0.4053, 'rouge2': 0.1857, 'rougeL': 0.2083, 'rougeLsum': 0.2083}",0.882200,-0.759700,13.105000
9,0.699300,2.379417,"{'rouge1': 0.4158, 'rouge2': 0.1926, 'rougeL': 0.2056, 'rougeLsum': 0.2056}",0.878900,-0.762000,13.730000
10,0.703300,2.444956,"{'rouge1': 0.4119, 'rouge2': 0.1928, 'rougeL': 0.2059, 'rougeLsum': 0.2059}",0.880400,-0.761100,13.165000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer is attempting to log a value of "{'rouge1': 0.4472, 'rouge2': 0.1969, 'rougeL': 0.1958, 'rougeLsum': 0.1958}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer is attempting to log a value of "{'rouge1': 0.3973, 'rouge2': 0.1803, 'rougeL': 0.1995, 'rougeLsum': 0.1995}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

events.out.tfevents.1705340851.5cb99e3abb0d.930.0:   0%|          | 0.00/39.4k [00:00<?, ?B/s]

events.out.tfevents.1705347656.5cb99e3abb0d.930.1:   0%|          | 0.00/517 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sarahahatee/t5-summarization-zero-shot-headers-and-better-prompt-base-enriched/commit/13616b4ae6cdb131c17de7ca0a7a0ba4fc3a2ac1', commit_message='End of training', commit_description='', oid='13616b4ae6cdb131c17de7ca0a7a0ba4fc3a2ac1', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
# to save a file to use with the task's evaluation python script
import csv
from transformers import pipeline

pipe = pipeline("text2text-generation", model="sarahahatee/t5-summarization-zero-shot-headers-and-better-prompt-base-enriched")

# Create a CSV file to store the outputs
output_file = 'summarization_output_zero_shot_better_prompt_base_enriched.csv'

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['ID', 'dialogue', 'section_text'])  # Writing headers

    for example in test_dataset:
        summary_text = pipe(f"{PREFIX_SUMMARIZE} {example['section_header']}: {example['dialogue']}")[0]['generated_text']
        print(summary_text)
        csv_writer.writerow([example['ID'], example['dialogue'], summary_text])


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


CC.


Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


FAM/SOCHX: A 2-1/2-year-old son and 5-1/2-
This 51-year-old white male who presented to the office complaining of progressive lymph node
ALLERGY
COMPLICATIONS: 200 mL of Macrodantin daily for urinary tract
PASTMEDICALHX
INTEGUMENTARY: Negative shortness of breath, negative snoring
PASTSURGICAL. The patient had a testicles removed in the summer of
PASTMEDICALHX: Left shoulder injury. GENERAL: No GERD
PASTMEDICALHX
PASTMEDICALHX. The patient has a history of Huntington disease and 
Noncontributory.
Lives at Life Care Center. Denied tobacco/ETOH/illicit drug use.
None.
He is married. He has a 15-year history of tobacco use. He smokes
GYNHX.
83-year-old female with bilateral hand weakness. She presents today with a 
CONDITION.
PASTMEDICALHX: Recently had pneumonia, URI, uroseps
CODE STATUS: Hydrocarbon aspiration pneumonia. No spleen.
Probably congestive.
1989.
PASTMEDICALHX.
MSK: The node has been swollen for two days.
PASTMEDICALHX
Ms. A lives alone. Her husband passed away in 2005. She 